In [18]:
import os
import sys

import pandas as pd
from datetime import date

# Required to import other modules from this project, in folders such as utils/ or notebooks/
current_folder = os.getcwd()
project_root_folder = os.path.abspath(os.path.join(current_folder, ".."))
sys.path.append(project_root_folder)
# Import functionalities from other modules in this project
from utils.db_utils import *

# Load .env file for AWS RDS login credentials
from dotenv import load_dotenv
dotenv_path = os.path.join(project_root_folder, ".env")
load_dotenv(dotenv_path)

# Global variables
conn, cursor = connect_to_rds()

✅ Connected successfully!


### Extract from staging table

In [19]:
# Test data
start_date = '2025-06-02'
end_date = '2025-06-03'
tickers = ["SPY", "GLD"]
ticker_list_as_str = "'" + "','".join(ticker for ticker in tickers) + "'"

In [20]:
ticker_list_as_str

"'SPY','GLD'"

In [ ]:
def extract_raw_data_from_staging(start_date: date, end_date: date, tickers: List[str], cursor: Cursor, conn: Connection):
    """
    Given start_date, end_date, and list of tickers, extract raw/unadjusted price & volume data from tbl_tiingo_daily_staging, as a Pandas dataframe
    """

    # Example: 'SPY','GLD'
    ticker_list_as_str = "'" + "','".join(ticker for ticker in tickers) + "'"

    query = f"""
    select
        ticker, business_date,
        open, high, low, close, volume,
        div_cash, split_factor
    from tbl_tiingo_daily_staging
    where business_date between '{start_date}' and '{end_date}' and ticker in ({ticker_list_as_str})
    order by ticker, business_date;
    """

    df = sql_query_as_df(query, cursor)
    return df

In [ ]:
extract_raw_data_from_staging(start_date, end_date, tickers, cursor, conn)


    select
        ticker, business_date,
        open, high, low, close, volume,
        div_cash, split_factor
    from tbl_tiingo_daily_staging
    where business_date between '2025-06-02' and '2025-06-03' and ticker in ('SPY','GLD')
    order by ticker, business_date;
    


,ticker,business_date,open,high,low,close,volume,div_cash,split_factor
0,GLD,2025-06-02,309.21,311.72,308.72,311.67,13593232,0.0,1.0
1,GLD,2025-06-03,309.56,309.56,307.15,308.91,9315081,0.0,1.0
2,SPY,2025-06-02,587.76,592.79,585.06,592.71,61630502,0.0,1.0
3,SPY,2025-06-03,592.34,597.08,591.85,596.09,63606204,0.0,1.0


### Adjust for corporate actions for a single stock

In [24]:
# Create example dataframe of raw prices + corporate actions, where the price moves strictly due to corporate actions and actually has zero returns every day for an investor of the stock
df = pd.DataFrame({
    "business_date": pd.to_datetime([
        "2025-06-01", "2025-06-02", "2025-06-03", "2025-06-04", "2025-06-05", "2025-06-06", "2025-06-07", "2025-06-08"
    ]),
    "raw_price": [306, 306, 300, 100, 100, 98, 49, 49],
    "div_cash": [0, 0, 6, 0, 0, 2, 0, 0],
    "split_factor": [1, 1, 1, 3, 1, 1, 2, 1]
})

In [25]:
df["adj_price"] = df["raw_price"].copy()

In [26]:
# Test works: we prove that the adjusted price for this stock is $49 the whole way through
for i in range(len(df)):
    dividend = df.iloc[i]["div_cash"]
    split_ratio = df.iloc[i]["split_factor"]
    
    if dividend > 0:
        df.loc[:i-1, "adj_price"] = df.loc[:i-1, "adj_price"] - dividend
    
    if split_ratio != 1:
        df.loc[:i-1, "adj_price"] = df.loc[:i-1, "adj_price"] / split_ratio
    
    print("i:",i)
    print("dividend:",dividend)
    print("split_ratio:",split_ratio)
    print(df)
    print()

i: 0
dividend: 0
split_ratio: 1
  business_date  raw_price  div_cash  split_factor  adj_price
0    2025-06-01        306         0             1        306
1    2025-06-02        306         0             1        306
2    2025-06-03        300         6             1        300
3    2025-06-04        100         0             3        100
4    2025-06-05        100         0             1        100
5    2025-06-06         98         2             1         98
6    2025-06-07         49         0             2         49
7    2025-06-08         49         0             1         49

i: 1
dividend: 0
split_ratio: 1
  business_date  raw_price  div_cash  split_factor  adj_price
0    2025-06-01        306         0             1        306
1    2025-06-02        306         0             1        306
2    2025-06-03        300         6             1        300
3    2025-06-04        100         0             3        100
4    2025-06-05        100         0             1        100
5    

In [ ]:
# Show final result
df

,business_date,raw_price,div_cash,split_factor,adj_price
0,2025-06-01,306,0,1,49
1,2025-06-02,306,0,1,49
2,2025-06-03,300,6,1,49
3,2025-06-04,100,0,3,49
4,2025-06-05,100,0,1,49
5,2025-06-06,98,2,1,49
6,2025-06-07,49,0,2,49
7,2025-06-08,49,0,1,49


### Try another example

In [43]:
# Create example dataframe of raw prices + corporate actions, where the price moves strictly due to corporate actions and actually has zero returns every day for an investor of the stock
df = pd.DataFrame({
    "business_date": pd.to_datetime([
        "2025-06-01", "2025-06-02", "2025-06-03", "2025-06-04", "2025-06-05", "2025-06-06", "2025-06-07", "2025-06-08"
    ]),
    "raw_price": [100, 99, 101, 50, 49, 50, 51, 25],
    "div_cash": [0, 1, 0, 0, 1, 0, 0, 0],
    "split_factor": [1, 1, 1, 2, 1, 1, 1, 2]
})

In [44]:
df["adj_price"] = df["raw_price"].copy()

In [45]:
# Test works: we prove that the adjusted price for this stock is $49 the whole way through
for i in range(len(df)):
    dividend = df.iloc[i]["div_cash"]
    split_ratio = df.iloc[i]["split_factor"]
    
    if dividend > 0:
        df.loc[:i-1, "adj_price"] = df.loc[:i-1, "adj_price"] - dividend
    
    if split_ratio != 1:
        df.loc[:i-1, "adj_price"] = df.loc[:i-1, "adj_price"] / split_ratio
    
    print("i:",i)
    print("dividend:",dividend)
    print("split_ratio:",split_ratio)
    print(df)
    print()

i: 0
dividend: 0
split_ratio: 1
  business_date  raw_price  div_cash  split_factor  adj_price
0    2025-06-01        100         0             1        100
1    2025-06-02         99         1             1         99
2    2025-06-03        101         0             1        101
3    2025-06-04         50         0             2         50
4    2025-06-05         49         1             1         49
5    2025-06-06         50         0             1         50
6    2025-06-07         51         0             1         51
7    2025-06-08         25         0             2         25

i: 1
dividend: 1
split_ratio: 1
  business_date  raw_price  div_cash  split_factor  adj_price
0    2025-06-01        100         0             1         99
1    2025-06-02         99         1             1         99
2    2025-06-03        101         0             1        101
3    2025-06-04         50         0             2         50
4    2025-06-05         49         1             1         49
5    

/tmp/ipykernel_1387/1723801590.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[49.5 49.5 50.5]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[:i-1, "adj_price"] = df.loc[:i-1, "adj_price"] / split_ratio


### Adjust for corporate actions for multiple stocks

In [90]:
df = pd.DataFrame({
    "ticker":["ABC", "ABC", "ABC", "ABC", "ABC", "ABC", "ABC", "ABC",
              "XYZ", "XYZ", "XYZ", "XYZ", "XYZ", "XYZ", "XYZ", "XYZ"],
    "business_date": pd.to_datetime([
        "2025-06-01", "2025-06-02", "2025-06-03", "2025-06-04", "2025-06-05", "2025-06-06", "2025-06-07", "2025-06-08",
        "2025-06-01", "2025-06-02", "2025-06-03", "2025-06-04", "2025-06-05", "2025-06-06", "2025-06-07", "2025-06-08"]),
    "raw_price": [306, 306, 300, 100, 100, 98, 49, 49,
                  100, 99, 101, 50, 49, 50, 51, 25],
    "div_cash": [0, 0, 6, 0, 0, 2, 0, 0,
                 0, 1, 0, 0, 1, 0, 0, 0],
    "split_factor": [1, 1, 1, 3, 1, 1, 2, 1,
                     1, 1, 1, 2, 1, 1, 1, 2]
})

In [91]:
df

,ticker,business_date,raw_price,div_cash,split_factor
0,ABC,2025-06-01,306,0,1
1,ABC,2025-06-02,306,0,1
2,ABC,2025-06-03,300,6,1
3,ABC,2025-06-04,100,0,3
4,ABC,2025-06-05,100,0,1
5,ABC,2025-06-06,98,2,1
6,ABC,2025-06-07,49,0,2
7,ABC,2025-06-08,49,0,1
8,XYZ,2025-06-01,100,0,1
9,XYZ,2025-06-02,99,1,1


In [95]:
def adj_corp_actions_for_one_stock(df_ticker: pd.DataFrame) -> pd.DataFrame:

    # Make a copy to avoid mutating the original dataframe
    df_ticker = df_ticker.copy()

    # Reset index of df_ticker to start from 0. Otherwise, if we don't do this, we will inherit index of df, and this iterative algorithm will fail for stock 2 onwards, and only work for the first stock in df
    df_ticker = df_ticker.reset_index(drop=True)

    # Cast to float to avoid issues with division and subtract later on
    df_ticker["adj_price"] = df_ticker["raw_price"].astype(float)
    
    # Iterate through every business_date for this stock, from past to present
    for i in df_ticker.index:
        
        dividend = df_ticker.loc[i,"div_cash"]
        split_ratio = df_ticker.loc[i,"split_factor"]
        
        # If there is a dividend payment today, subtract all prices from yesterday & before by dividend amount to make history comparable to today
        if dividend > 0:
            df_ticker.loc[:i-1, "adj_price"] -= dividend
        
        # If there is a stock split today, divide all prices from yesterday & before by split_ratio to make history comparable to today
        if split_ratio != 1:
            df_ticker.loc[:i-1, "adj_price"] /= split_ratio
    
    return df_ticker

In [96]:
# group_keys = False to avoid multi-layer index, i.e. - to keep rows keyed on (ticker, business_date)
# include_groups = True to include ticker as a column in the resulting output dataframe. If false, ticker will not appear in result
def adj_corp_actions(df: pd.DataFrame) -> pd.DataFrame:
    result = df.groupby("ticker", group_keys = False).apply(adj_corp_actions_for_one_stock, include_groups = True)
    return result

In [97]:
adj_corp_actions(df)

/tmp/ipykernel_1380/3781747306.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result = df.groupby("ticker", group_keys = False).apply(adj_corp_actions_for_one_stock, include_groups = True)


,ticker,business_date,raw_price,div_cash,split_factor,adj_price
0,ABC,2025-06-01,306,0,1,49.00
1,ABC,2025-06-02,306,0,1,49.00
2,ABC,2025-06-03,300,6,1,49.00
3,ABC,2025-06-04,100,0,3,49.00
4,ABC,2025-06-05,100,0,1,49.00
5,ABC,2025-06-06,98,2,1,49.00
6,ABC,2025-06-07,49,0,2,49.00
7,ABC,2025-06-08,49,0,1,49.00
0,XYZ,2025-06-01,100,0,1,24.25
1,XYZ,2025-06-02,99,1,1,24.25
